In [2]:
import sys
from pathlib import Path


project_root = Path.cwd().parent


if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.text_utils import crear_input_oferta, crear_input_cv
from sentence_transformers import SentenceTransformer, util
import pandas as pd

model = SentenceTransformer('all-MiniLM-L6-v2')
print("Rutas configuradas y módulos importados correctamente.")

c:\Users\zveng\Desktop\TFG\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zveng\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to r

Rutas configuradas y módulos importados correctamente.


In [4]:
# Carga de los datos procesados
try:
    offers_df = pd.read_csv(project_root / 'data/01_processed/offers_processed.csv')
    cvs_df = pd.read_csv(project_root / 'data/01_processed/cvs_processed.csv')
    print("Datos cargados correctamente.")
except FileNotFoundError as e:
    print(f"Error: No se pudo encontrar el archivo {e.filename}. Asegúrate de que la estructura de carpetas es correcta.")

# Carga del modelo de embeddings para el baseline
print("Cargando modelo de embeddings...")
baseline_model = SentenceTransformer('all-MiniLM-L6-v2')
print("Modelo baseline cargado.")

Datos cargados correctamente.
Cargando modelo de embeddings...
Modelo baseline cargado.


In [5]:
def get_baseline_score(offer_row, cv_row, model):
    """
    Calcula la similitud del coseno entre una oferta y un CV usando un modelo de SentenceTransformer.
    
    Args:
        offer_row (pd.Series): Una fila del DataFrame de ofertas.
        cv_row (pd.Series): Una fila del DataFrame de CVs.
        model (SentenceTransformer): El modelo de embeddings cargado.
        
    Returns:
        float: El score de similitud del coseno (entre -1 y 1).
    """
    # 1. Formatear los textos de entrada usando las utilidades de 'src'
    offer_text = crear_input_oferta(offer_row)
    cv_text = crear_input_cv(cv_row)
    
    # 2. Generar los embeddings (vectores)
    offer_embedding = model.encode(offer_text, convert_to_tensor=True)
    cv_embedding = model.encode(cv_text, convert_to_tensor=True)
    
    # 3. Calcular y devolver la similitud del coseno
    cosine_score = util.cos_sim(offer_embedding, cv_embedding)
    
    return cosine_score.item()

In [6]:
# Coge un ejemplo para probar que todo funciona
sample_offer = offers_df.iloc[0]
sample_cv = cvs_df.iloc[0]

# Calcula el score
score = get_baseline_score(sample_offer, sample_cv, baseline_model)

print("--- Prueba del Baseline ---")
print(f"Oferta de ejemplo (ID): {sample_offer['job_id']}")
print(f"CV de ejemplo (ID): {sample_cv['candidate_id']}")
print(f"\nScore de Similitud del Coseno: {score:.4f}")

--- Prueba del Baseline ---
Oferta de ejemplo (ID): 921716
CV de ejemplo (ID): cand_0

Score de Similitud del Coseno: 0.3315
